# Figure 4

In [ ]:
#Import relevant packages
import numpy as np
import pandas as pd
from matplotlib import rcParams
import os
import scanpy as sc

import matplotlib as mpl
import matplotlib.pyplot as plt

import cmocean

import seaborn as sns

from scipy.stats import median_abs_deviation

import anndata as ad

#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

#Set fontsize
plt.rcParams.update({'font.size': 20})

In [ ]:
adata_OE = sc.read_h5ad('/hpc/group/goldsteinlab/vmd13/Python/251022_AD_22_samples_+qc_scVI_5.0.h5ad')

In [ ]:
adata_OE.obs.cluster_map

In [ ]:
#subet neuron only
neuron_mask = adata_OE.obs["cluster_map"] == "neuron"
adata_OE = adata_OE[neuron_mask].copy() 
adata_OE

In [ ]:
adata = adata_OE

In [ ]:
#Prep for HVG and scvi
# create normalized layer and log1p in .obs

#log1p the data
adata.obs["log1p_total_counts"] = np.log1p(adata.obs["total_counts"])

#Create normalized layers
adata.layers["counts"] = adata.X.copy()
adata.layers['norm'] = adata.X.copy(); sc.pp.normalize_total(adata, target_sum=1e4, layer="norm") # this is relative counts normalized per cell

In [ ]:
#HVG via Scanpy
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
)

In [ ]:
adata.var['mean_'] = np.array(adata.X.mean(0))[0]
adata.var['frac_zero'] = 1 - np.array((adata.X > 0).sum(0))[0] / adata.shape[0]

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

ax.scatter(adata.var.mean_, adata.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata, n_top_genes=2000, inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata.varm['df_poisson']= df_poisson

adata_query = adata[:, is_hvg].copy()
print(adata_query)

In [ ]:
#Set up scvi model

scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key="orig_patients",
    continuous_covariate_keys=["pct_counts_mt"],
)

model = scvi.model.SCVI(adata_query, gene_likelihood="nb")

model.view_anndata_setup()

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Plot model results
train_elbo = model.history['elbo_train'][1:]
test_elbo = model.history['elbo_validation']

ax = train_elbo.plot()
test_elbo.plot(ax = ax)

In [ ]:
# Fit model to data
latent = model.get_latent_representation()
adata.obsm["X_scVI_1.1_neuron"] = latent

# Calculate neighbors using scVI latent representation
sc.pp.neighbors(adata, use_rep="X_scVI_1.1_neuron")
sc.tl.umap(adata, min_dist=0.5)

# Run Leiden clustering at multiple resolutions
resolutions = [1.0, 3.0]
for res in resolutions:
    sc.tl.leiden(adata, key_added=f"leiden_scVI_1.1_neuron_res{res}", resolution=res)


In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
sc.pl.umap(adata, color="leiden_scVI_1.1_neuron_res3.0", legend_loc="on data", color_map="cmo.matter", vmax="p95", layer="norm", ax=ax, s=100, frameon=False, save=False)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
sc.pl.umap(adata, color="orig_patients", legend_loc="right margin", color_map="cmo.matter", vmax="p95", layer="norm", ax=ax, s=100, frameon=False, save=False)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mt", "log1p_total_counts", "orig_patients", "Alz_status"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

## neuron classification 

In [ ]:
sc.pl.umap(adata, color="leiden_scVI_1.1_neuron_res3.0", legend_loc="on data", s=80, frameon=False, save=False)

In [ ]:
#examining cell type markers

markers_dict = {}
markers_dict["GBC/INP"] = ["ASCL1","NEUROD1"]
markers_dict["iOSN"] = ["LHX2", "GNG8"]
markers_dict["mOSN"] = ["GNG13", "STOML3"]


for cell_type in markers_dict:
    print("examining",cell_type,"markers\n")
    sc.pl.umap(
    adata,
    color=markers_dict[cell_type],
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.matter",
    ncols=2,
    frameon=False,
    vmax="p90",
    layer="norm",
    save=False
)

In [ ]:
to_drop = {'5', '7', '10', '1', '30', '21'}

mask   = ~adata.obs['leiden_scVI_1.1_neuron_res3.0'].astype(str).isin(to_drop)
adata  = adata[mask].copy()

if adata.obs['leiden_scVI_1.1_neuron_res3.0'].dtype.name == 'category':
    adata.obs['leiden_scVI_1.1_neuron_res3.0'] = (
        adata.obs['leiden_scVI_1.1_neuron_res3.0'].cat.remove_unused_categories()
    )


In [ ]:
sc.pl.umap(adata, color="leiden_scVI_1.1_neuron_res1.0", legend_loc="on data", s=80, frameon=False, save=False)

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scVI_1.1_neuron")
sc.tl.umap(adata, min_dist=0.5)

resolutions = [1.0, 3.0]
for res in resolutions:
    sc.tl.leiden(adata, key_added=f"leiden_scVI_1.1_neuron_res{res}", resolution=res)


In [ ]:
sc.pl.umap(adata, color="leiden_scVI_1.1_neuron_res1.0", legend_loc="on data", s=80, frameon=False, save=False)

In [ ]:
to_drop = {'12', '6'}

mask   = ~adata.obs['leiden_scVI_1.1_neuron_res1.0'].astype(str).isin(to_drop)
adata  = adata[mask].copy()

if adata.obs['leiden_scVI_1.1_neuron_res1.0'].dtype.name == 'category':
    adata.obs['leiden_scVI_1.1_neuron_res1.0'] = (
        adata.obs['leiden_scVI_1.1_neuron_res1.0'].cat.remove_unused_categories()
    )


In [ ]:
sc.pl.umap(adata, color="leiden_scVI_1.1_neuron_res1.0", legend_loc="on data", s=80, frameon=False, save=False)

In [ ]:
cluster2name = {
     13 : "INP", 9 : "INP",
     0 : "mOSN", 2 : "mOSN", 7 : "mOSN",
     11 : "iOSN", 4 : "iOSN", 3 : "iOSN", 8 : "iOSN", 
     10 : "iOSN", 1 : "iOSN", 5 : "iOSN", 6 : "iOSN", 
}

adata.obs["neuron_names"] = (
    adata.obs["leiden_scVI_1.1_neuron_res1.0"]
         .astype(int)         
         .map(cluster2name)   
         .astype("category")  
)

adata.obs["neuron_names"].cat.reorder_categories(
    ["INP", "iOSN", "mOSN"],
    ordered=True,
    inplace=True
)


In [ ]:
sc.pl.umap(adata, color="neuron_names", legend_loc="on data", s=80, frameon=False, save=False)

In [ ]:
sc.pl.umap(adata, color="orig_patients", legend_loc="right margin", s=80, frameon=False, save=False)

In [ ]:
to_drop = {'6'}

mask   = ~adata.obs['leiden_scVI_1.1_neuron_res1.0'].astype(str).isin(to_drop)
adata  = adata[mask].copy()

if adata.obs['leiden_scVI_1.1_neuron_res1.0'].dtype.name == 'category':
    adata.obs['leiden_scVI_1.1_neuron_res1.0'] = (
        adata.obs['leiden_scVI_1.1_neuron_res1.0'].cat.remove_unused_categories()
    )


In [ ]:
to_drop = {'7'}

mask   = ~adata.obs['leiden_scVI_1.1_neuron_res1.0'].astype(str).isin(to_drop)
adata  = adata[mask].copy()

if adata.obs['leiden_scVI_1.1_neuron_res1.0'].dtype.name == 'category':
    adata.obs['leiden_scVI_1.1_neuron_res1.0'] = (
        adata.obs['leiden_scVI_1.1_neuron_res1.0'].cat.remove_unused_categories()
    )


In [ ]:
sc.pl.umap(adata, color="orig_patients", legend_loc="right margin", s=80, frameon=False, save=False)

In [ ]:
adata.layers["log_norm"] = adata.layers["counts"] 
sc.pp.normalize_total(adata, target_sum=1e4, layer="log_norm")
sc.pp.log1p(adata,               layer="log_norm")


In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc

markers_dict = {
    "INP": ["NEUROD1"],
    "iOSN": ["LHX2", "GNG8"],
    "mOSN": ["GNG13", "STOML3"],
}

sc.pl.dotplot(
    adata,
    var_names=markers_dict,
    groupby="neuron_names",
    layer="log_norm",
    standard_scale="var",
    dot_max=1.0,
    show=False,
)

fig = plt.gcf()
fig.set_size_inches(5.0, 2.4)
plt.setp(fig.axes[0].get_xticklabels(), rotation=45, ha="right")

leg = fig.axes[-1].legend_            
if leg is not None:
    leg.set_bbox_to_anchor((1.25, 0.5))   
    leg._legend_box.sep = 80               
    leg.set_title("Dot size\n(cell %)", prop={"weight": "bold"})
    for txt in leg.get_texts():
        txt.set_fontsize(8)

plt.tight_layout()
fig.savefig("/work/vmd13/250710_dotplot_neurons.svg", format="svg",
             bbox_inches="tight", dpi=300)
plt.show()


## neuron pseudobulk

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox
import anndata


import os
os.environ['R_HOME']='/hpc/group/goldsteinlab/envs/vmd13_Python_R_4_env/lib/R'

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
#Show specific size of pandas dataframe when produced
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
%%R
.libPaths( c( "/hpc/group/goldsteinlab/envs/vmd13_Python_R_4_env/lib/R" , .libPaths() ) )

In [ ]:
%%R
setwd('/work/vmd13')

In [ ]:
%%R
library("Rcpp")
library("ggplot2")
library("ggrepel")
library("repr")
library('edgeR')

In [ ]:
cell_counts = (
    adata.obs["orig_patients"]
         .value_counts()     
         .sort_index()       
)

print(cell_counts)


In [ ]:
import os
import hashlib

import numpy as np
import pandas as pd
from scipy import sparse

N_CELLS = 9
N_REPS = 3
OUTDIR = "/hpc/group/goldsteinlab/vmd13/Python/250710_neuron_bulks_v3"
master_seed = 42

os.makedirs(OUTDIR, exist_ok=True)

mat = adata.layers["raw"] if "raw" in adata.layers else adata.X
genes = adata.var_names.to_numpy()

def seed_for_patient(pid, master):
    h = hashlib.md5(f"{master}|{pid}".encode("utf-8")).digest()
    return int.from_bytes(h[:4], "little")

cols, meta = [], []

groups = adata.obs.groupby("orig_patients", sort=True).groups
for pid, idx in groups.items():
    idx_int = adata.obs_names.get_indexer(idx)
    n_tot = idx_int.size
    if n_tot < N_CELLS * N_REPS:
        print(f"· skip {pid}: only {n_tot} cells")
        continue

    rng = np.random.default_rng(seed_for_patient(str(pid), master_seed))
    sel = rng.permutation(idx_int)[: N_CELLS * N_REPS]
    rep_blocks = np.split(sel, N_REPS)

    for r, rep_idx in enumerate(rep_blocks, 1):
        bulk_id = f"{pid}_rep{r}"

        if sparse.issparse(mat):
            counts = np.asarray(mat[rep_idx].sum(axis=0)).ravel()
            lib_umis = int(mat[rep_idx].sum())
        else:
            counts = np.asarray(mat[rep_idx].sum(axis=0)).ravel()
            lib_umis = int(mat[rep_idx].sum())

        cols.append(counts)
        meta.append(
            dict(
                bulk_id=bulk_id,
                orig_patient=pid,
                replicate=r,
                Alz_status=adata.obs.loc[idx[0], "Alz_status"],
                n_cells=int(rep_idx.size),
                lib_umis=lib_umis,
            )
        )

counts_df = pd.DataFrame(np.vstack(cols).T, index=genes, columns=[m["bulk_id"] for m in meta])
meta_df = pd.DataFrame(meta).set_index("bulk_id")

counts_df.to_csv(os.path.join(OUTDIR, "pseudobulk_counts.csv"))
meta_df.to_csv(os.path.join(OUTDIR, "pseudobulk_meta.csv"))

print(f"✔ saved {counts_df.shape[1]} pseudobulks from {meta_df['orig_patient'].nunique()} patients → {OUTDIR}")

In [ ]:
%%R
suppressPackageStartupMessages({
  library(edgeR)
  library(limma)
})

indir <- "/hpc/group/goldsteinlab/vmd13/Python/250710_neuron_bulks_v3"

counts <- read.csv(file.path(indir, "pseudobulk_counts.csv"),
                   row.names = 1, check.names = FALSE)
meta <- read.csv(file.path(indir, "pseudobulk_meta.csv"),
                 row.names = 1, check.names = FALSE)

meta <- meta[colnames(counts), , drop = FALSE]

na_bulk <- rownames(meta)[is.na(meta$Alz_status)]
if (length(na_bulk)) {
  counts <- counts[, !colnames(counts) %in% na_bulk, drop = FALSE]
  meta <- meta[!rownames(meta) %in% na_bulk, , drop = FALSE]
}

y <- DGEList(counts = as.matrix(counts), samples = meta)
y$samples$Alz_status <- factor(gsub("-", "", y$samples$Alz_status),
                               levels = c("Control", "PreClinical", "Clinical"))

design <- model.matrix(~ 0 + Alz_status, data = y$samples)
colnames(design) <- levels(y$samples$Alz_status)

keep <- filterByExpr(y, design = design)
y <- y[keep, , keep.lib.sizes = FALSE]
y <- calcNormFactors(y)
y <- estimateDisp(y, design)
fit <- glmQLFit(y, design)

con_PC <- makeContrasts(PreClinical - Control, levels = design)
con_CL <- makeContrasts(Clinical - Control, levels = design)

de_PC <- topTags(glmQLFTest(fit, contrast = con_PC), n = Inf)$table
de_CL <- topTags(glmQLFTest(fit, contrast = con_CL), n = Inf)$table

write.csv(de_PC, file.path(indir, "DE_preclinical_vs_control.csv"))
write.csv(de_CL, file.path(indir, "DE_clinical_vs_control.csv"))

cat("✓ finished —", nrow(design), "bulks analysed; results in", indir, "\n")

In [ ]:
%%R
indir <- "/hpc/group/goldsteinlab/vmd13/Python/250710_neuron_bulks_v3"

de_PC <- read.csv(file.path(indir, "DE_preclinical_vs_control.csv"), row.names = 1)
de_CL <- read.csv(file.path(indir, "DE_clinical_vs_control.csv"), row.names = 1)

In [ ]:
%%R
suppressPackageStartupMessages({
  library(ggplot2)
  library(ggrepel)
})

sig_cut <- 0.05
lab_cut <- 1.0
axis_cap <- 3

out_pdf <- "/work/vmd13/quadrant_DE_preclin_clinical_crop3_neuron.pdf"
out_png <- "/work/vmd13/quadrant_DE_preclin_clinical_crop3_neuron.png"

both <- merge(de_PC[, c("logFC", "FDR")],
              de_CL[, c("logFC", "FDR")],
              by = "row.names", suffixes = c("_PC", "_CL"))
colnames(both)[1] <- "gene"

both <- both[!grepl("^UTY$|\\.|^ENSG|^MT-|^H2|^H3", both$gene, perl = TRUE), ]

both$cat <- "Not sig"
both$cat[both$FDR_PC < sig_cut & both$FDR_CL < sig_cut & both$logFC_PC > 0 & both$logFC_CL > 0] <- "Up in both"
both$cat[both$FDR_PC < sig_cut & both$FDR_CL < sig_cut & both$logFC_PC < 0 & both$logFC_CL < 0] <- "Down in both"
both$cat[both$FDR_PC < sig_cut & both$FDR_CL >= sig_cut & both$logFC_PC > 0] <- "Up PC only"
both$cat[both$FDR_PC < sig_cut & both$FDR_CL >= sig_cut & both$logFC_PC < 0] <- "Down PC only"
both$cat[both$FDR_CL < sig_cut & both$FDR_PC >= sig_cut & both$logFC_CL > 0] <- "Up CL only"
both$cat[both$FDR_CL < sig_cut & both$FDR_PC >= sig_cut & both$logFC_CL < 0] <- "Down CL only"

both$cat <- factor(both$cat,
                   levels = c("Up in both", "Down in both",
                              "Up PC only", "Down PC only",
                              "Up CL only", "Down CL only", "Not sig"))

pal <- c("Up in both" = "#e41a1c",
         "Down in both" = "#377eb8",
         "Up PC only" = "#984ea3",
         "Down PC only" = "#4daf4a",
         "Up CL only" = "#ff7f00",
         "Down CL only" = "#a65628",
         "Not sig" = "grey75")

both$label <- ifelse(
  (abs(both$logFC_PC) >= lab_cut | abs(both$logFC_CL) >= lab_cut) &
    (both$FDR_PC < sig_cut | both$FDR_CL < sig_cut),
  both$gene, NA
)

p <- ggplot(both, aes(x = logFC_PC, y = logFC_CL, colour = cat)) +
  geom_hline(yintercept = 0, colour = "grey70") +
  geom_vline(xintercept = 0, colour = "grey70") +
  geom_point(size = 1.2, alpha = 0.8) +
  geom_text_repel(aes(label = label),
                  max.overlaps = 800,
                  box.padding = 0.3,
                  point.padding = 0.2,
                  min.segment.length = 0,
                  size = 2.6,
                  seed = 42) +
  scale_colour_manual(values = pal, name = NULL, drop = FALSE) +
  coord_cartesian(xlim = c(-axis_cap, axis_cap), ylim = c(-axis_cap, axis_cap)) +
  labs(x = "log2 FC  (Pre-Clinical − Control)",
       y = "log2 FC  (Clinical − Control)",
       title = "Quadrant plot of differential expression") +
  theme_classic() +
  theme(legend.position = "right")

ggsave(out_pdf, plot = p, width = 6, height = 6, units = "in")
ggsave(out_png, plot = p, width = 6, height = 6, units = "in", dpi = 300)

print(p)

In [ ]:
%%R
n_up_both   <- sum(both$cat == "Up in both")
n_down_both <- sum(both$cat == "Down in both")

cat("\nConcordant DE genes (FDR <", sig_cut, "in both contrasts):\n",
    "  Up in both   :", n_up_both, "\n",
    "  Down in both :", n_down_both, "\n")


In [ ]:
%%R
up_both_genes   <- both$gene[both$cat == "Up in both"]
down_both_genes <- both$gene[both$cat == "Down in both"]

cat("\nTop concordant up-regulated genes:\n")
print(head(up_both_genes, 20))   # show first 20
cat("\nTop concordant down-regulated genes:\n")
print(head(down_both_genes, 20))

In [ ]:
%%R
suppressPackageStartupMessages({
  library(edgeR)
  library(ggplot2)
  library(reshape2)
})

logCPM <- cpm(y, log = TRUE, prior.count = 2)

genes <- c("AKR1B1", "CERT1")
present <- intersect(genes, rownames(logCPM))

expr_df <- as.data.frame(t(logCPM[present, , drop = FALSE]))
expr_df$Alz_status <- y$samples$Alz_status

expr_long <- melt(expr_df, id.vars = "Alz_status",
                  variable.name = "Gene", value.name = "logCPM")

expr_long$Alz_status <- factor(expr_long$Alz_status,
                               levels = c("Control", "PreClinical", "Clinical"),
                               labels = c("Control", "Pre-Clinical", "Clinical"))

stage_pairs <- combn(levels(expr_long$Alz_status), 2, simplify = FALSE)

for (g in genes) {
  cat("\n", g, "\n", sep = "")
  gene_dat <- expr_long[expr_long$Gene == g, ]
  p_raw <- sapply(stage_pairs, function(pr) {
    x <- gene_dat$logCPM[gene_dat$Alz_status == pr[1]]
    y <- gene_dat$logCPM[gene_dat$Alz_status == pr[2]]
    wilcox.test(x, y, exact = FALSE)$p.value
  })
  p_adj <- p.adjust(p_raw, method = "BH")
  for (i in seq_along(stage_pairs)) {
    pr <- stage_pairs[[i]]
    cat(sprintf("  %s vs %s : raw p = %.3g  adj p = %.3g\n",
                pr[1], pr[2], p_raw[i], p_adj[i]))
  }
}

p <- ggplot(expr_long, aes(x = Alz_status, y = logCPM, fill = Alz_status)) +
  geom_boxplot(outlier.shape = NA, alpha = 0.8, width = 0.65) +
  scale_fill_manual(values = c("#bfbfbf", "#625c86", "#7295b4"), guide = "none") +
  facet_wrap(~ Gene, scales = "free_y") +
  labs(x = "", y = "log2 CPM", title = "Pseudobulk expression by Alzheimer stage") +
  theme_classic() +
  theme(strip.text = element_text(size = 11),
        axis.text.x = element_text(angle = 30, hjust = 1))

out_png <- "/work/vmd13/pseudobulk_AKR1B1_CERT1_boxplots.png"
ggsave(out_png, plot = p, width = 6, height = 4, units = "in", dpi = 300)

print(p)